# MIGMOBS

**Natalia Dedandi**



EXPORT DATA TO FLOURISH



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# prompt: open a csv file forma drive folder

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Replace 'your_file_path' with the actual path to your CSV file in Google Drive
file_path = '/content/drive/MyDrive/3. Proyectos/MIGMOBS_nat/3_data_repository/GT_Mobility_v1.0.csv'

mobility_df = pd.read_csv(file_path)


Mounted at /content/drive


In [3]:
# Replace 'your_file_path' with the actual path to your CSV file in Google Drive
file_path = '/content/drive/MyDrive/3. Proyectos/MIGMOBS_nat/3_data_repository/migration_imputed_RIKS_dec2021.csv'

df_stock_migrants = pd.read_csv(file_path)
print(df_stock_migrants.head())

  iso_or               origin iso_des destination  year  stock  flow  inflow  \
0    AAB  Antigua and Barbuda     ABW       Aruba  1960     16   NaN     NaN   
1    AAB  Antigua and Barbuda     ABW       Aruba  1961     16   0.0     NaN   
2    AAB  Antigua and Barbuda     ABW       Aruba  1962     15  -1.0     NaN   
3    AAB  Antigua and Barbuda     ABW       Aruba  1963     15   0.0     NaN   
4    AAB  Antigua and Barbuda     ABW       Aruba  1964     15   0.0     NaN   

   outflow  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  


In [4]:
countries = ['ARG', 'CHL']
latam_mig_df = df_stock_migrants[df_stock_migrants['iso_des'].isin(countries)]
latam_mig_df_grp = latam_mig_df.groupby(['year', 'iso_des'])['stock'].sum().reset_index()

# Pivotar los datos para obtener las columnas con los países como series
pivot_data = latam_mig_df_grp.pivot_table(index='year', columns='iso_des', values='stock').reset_index()
pivot_data

iso_des,year,ARG,CHL
0,1960,2683416.0,109563.0
1,1961,2630230.0,108385.0
2,1962,2636364.0,107452.0
3,1963,2656544.0,105956.0
4,1964,2667699.0,105497.0
...,...,...,...
56,2016,2101311.0,772580.0
57,2017,2145742.0,983752.0
58,2018,2183670.0,1207629.0
59,2019,2214059.0,1399076.0


In [5]:
pivot_data.to_csv('/content/drive/MyDrive/3. Proyectos/MIGMOBS_nat/3_data_repository/Flourish_data/stock_latam_migrants.csv', index=False)


In [6]:
file_path = '/content/drive/MyDrive/3. Proyectos/MIGMOBS_nat/3_data_repository/db_indicators.csv'

df_indicators = pd.read_csv(file_path)
print(df_indicators.head())

  country_code  year       value        indicator            fuente
0          ARG  2011  12133921.0  target_mobility  GT_Mobility_v1.0
1          BOL  2011   1493371.0  target_mobility  GT_Mobility_v1.0
2          CHL  2011   5916559.0  target_mobility  GT_Mobility_v1.0
3          VEN  2011   2849358.0  target_mobility  GT_Mobility_v1.0
4          ARG  2012  12678938.0  target_mobility  GT_Mobility_v1.0


In [7]:
#get population indicator
population_data = df_indicators[df_indicators['indicator'] == 'total_population_wb'][['country_code', 'year', 'value']].rename(columns={'value': 'total_population_wb'})

#target_iso3 country of destiny
population_data = population_data[population_data['country_code'].isin(countries)]

population_data


,country_code,year,total_population_wb
1180,CHL,1960,8153350.0
1181,ARG,1960,20386045.0
1184,CHL,1961,8324802.0
1185,ARG,1961,20726276.0
1188,CHL,1962,8497059.0
...,...,...,...
1417,ARG,2019,44973465.0
1420,CHL,2020,19370624.0
1421,ARG,2020,45191965.0
1424,CHL,2021,19456334.0


In [8]:

latam_mig_df_grp = latam_mig_df_grp.rename(columns={'iso_des': 'country_code'})
# Combinar los datos por país y año
merged_data = pd.merge(latam_mig_df_grp, population_data, on=['country_code', 'year'], how='inner')

# Calcular la nueva variable
merged_data['migration_proportion'] = merged_data['stock'] / merged_data['total_population_wb']*100

# Mostrar las primeras filas del resultado
print(merged_data.head())

   year country_code    stock  total_population_wb  migration_proportion
0  1960          ARG  2683416           20386045.0             13.163004
1  1960          CHL   109563            8153350.0              1.343779
2  1961          ARG  2630230           20726276.0             12.690316
3  1961          CHL   108385            8324802.0              1.301953
4  1962          ARG  2636364           21072538.0             12.510899


In [9]:
# Select the desired columns and pivot the data
pivot_migration = merged_data[['year', 'country_code', 'migration_proportion']].pivot(index='year', columns='country_code', values='migration_proportion').reset_index()

# Display the pivoted data
pivot_migration

country_code,year,ARG,CHL
0,1960,13.163004,1.343779
1,1961,12.690316,1.301953
2,1962,12.510899,1.264579
3,1963,12.401179,1.222079
4,1964,12.254323,1.193064
...,...,...,...
56,2016,4.786551,4.229324
57,2017,4.844876,5.300711
58,2018,4.890104,6.391874
59,2019,4.923034,7.287710


In [10]:
#export to flourish
pivot_migration.to_csv('/content/drive/MyDrive/3. Proyectos/MIGMOBS_nat/3_data_repository/Flourish_data/prop_latam_migrants.csv', index=False)

Agrego los datos de % de movilidad por población para graficar juntos



In [11]:
# prompt: show the differente values of df_indicators indicators variable

print(df_indicators['indicator'].unique())

['target_mobility' 'source_mobility_ARG' 'source_mobility_CHL'
 'inmigration_stock' 'inmigration_stock_INDEC' 'total_population_INDEC'
 'inmigration_stock_wb' 'inmigration_stock_oim' 'emigration_stock'
 'net_migration_calculated' 'net_migration_onu' 'total_population_wb'
 'population_onu' 'migration_pct_wb' 'migration_pct_calc' 'gdp_wb'
 'gini_wb' 'poverty_wb']


In [12]:
t_mobility = df_indicators[df_indicators['indicator'] == 'target_mobility'][['country_code', 'year', 'value']].rename(columns={'value': 'target_mobility'})

#target_iso3 country of destiny
t_mobility = t_mobility[t_mobility['country_code'].isin(countries)]
t_mobility

,country_code,year,target_mobility
0,ARG,2011,12133921.0
2,CHL,2011,5916559.0
4,ARG,2012,12678938.0
6,CHL,2012,6528705.0
8,ARG,2013,12500972.0
10,CHL,2013,6644472.0
12,ARG,2014,13044692.0
14,CHL,2014,7172922.0
16,ARG,2015,15021604.0
18,CHL,2015,8405899.0


In [13]:
# Combinar los datos por país y año
merged_data = pd.merge(t_mobility, population_data, on=['country_code', 'year'], how='inner')

# Calcular la nueva variable
merged_data['mobility_ratio'] = merged_data['target_mobility'] / merged_data['total_population_wb']*100

# Mostrar las primeras filas del resultado
print(merged_data.head())

  country_code  year  target_mobility  total_population_wb  mobility_ratio
0          ARG  2011       12133921.0           41730660.0       29.076753
1          CHL  2011        5916559.0           17351816.0       34.097636
2          ARG  2012       12678938.0           42161721.0       30.072155
3          CHL  2012        6528705.0           17519119.0       37.266172
4          ARG  2013       12500972.0           42582455.0       29.357096


In [14]:
# prompt: change value in gini_data  country_code=ARG as GINI-ARG and CHL as GINI-CHL

merged_data.loc[merged_data['country_code'] == 'ARG', 'country_code'] = 'MOV-ARG'
merged_data.loc[merged_data['country_code'] == 'CHL', 'country_code'] = 'MOV-CHL'


In [15]:
# Select the desired columns and pivot the data
pivot_mov = merged_data[['year', 'country_code', 'mobility_ratio']].pivot(index='year', columns='country_code', values='mobility_ratio').reset_index()

# Display the pivoted data
pivot_mov

country_code,year,MOV-ARG,MOV-CHL
0,2011,29.076753,34.097636
1,2012,30.072155,37.266172
2,2013,29.357096,37.566969
3,2014,30.319520,40.152506
4,2015,34.550682,46.576206
5,2016,38.701253,53.601662


In [16]:
# Combinar los datos por país y año
merged_mov_mig = pd.merge(pivot_mov, pivot_migration, on=['year'], how='inner')
merged_mov_mig

country_code,year,MOV-ARG,MOV-CHL,ARG,CHL
0,2011,29.076753,34.097636,4.389288,2.144882
1,2012,30.072155,37.266172,4.477246,2.274207
2,2013,29.357096,37.566969,4.557462,2.455413
3,2014,30.319520,40.152506,4.643338,2.775972
4,2015,34.550682,46.576206,4.705537,3.277933
5,2016,38.701253,53.601662,4.786551,4.229324


In [17]:
#export to flourish
merged_mov_mig.to_csv('/content/drive/MyDrive/3. Proyectos/MIGMOBS_nat/3_data_repository/Flourish_data/prop_mov_mig.csv', index=False)

In [18]:
#get population indicator
gini_data = df_indicators[df_indicators['indicator'] == 'gini_wb'][['country_code', 'year', 'value']].rename(columns={'value': 'gini_wb'})

#target_iso3 country of destiny
gini_data = gini_data[gini_data['country_code'].isin(countries)]



In [19]:
# prompt: change value in gini_data  country_code=ARG as GINI-ARG and CHL as GINI-CHL

gini_data.loc[gini_data['country_code'] == 'ARG', 'country_code'] = 'GINI-ARG'
gini_data.loc[gini_data['country_code'] == 'CHL', 'country_code'] = 'GINI-CHL'


In [20]:
# Select the desired columns and pivot the data
pivot_gini = gini_data[['year', 'country_code', 'gini_wb']].pivot(index='year', columns='country_code', values='gini_wb').reset_index()

# Display the pivoted data


In [21]:
# Combinar los datos por país y año
merged_gini_mig = pd.merge(pivot_gini, pivot_migration, on=['year'], how='inner')
merged_gini_mig

country_code,year,GINI-ARG,GINI-CHL,ARG,CHL
0,1980,40.8,NaN,3.486970,0.774555
1,1986,42.8,NaN,4.049357,0.786144
2,1987,45.3,56.2,4.292465,0.787184
3,1990,NaN,57.2,4.908859,0.829289
4,1991,46.8,NaN,4.824312,0.881442
5,1992,45.5,54.8,4.766150,0.913251
6,1993,44.8,NaN,4.656804,0.931509
7,1994,45.9,56.4,4.504807,0.971546
8,1995,48.9,NaN,4.348284,0.996002
9,1996,49.5,54.9,4.203775,1.009220


In [22]:
#export to flourish
merged_gini_mig.to_csv('/content/drive/MyDrive/3. Proyectos/MIGMOBS_nat/3_data_repository/Flourish_data/prop_gini_mig.csv', index=False)